<h1>Event Loop</h1>

using `root.after`- [documentation](https://tcl.tk/man/tcl8.6/TclCmd/after.htm) Here we show how we can inturrupt, but the more important is how we are looping instead of a for loopo we are calling `root.after()` and implicitly uping the count and giving it some time during this loop to simulate work being done. using `after()` is what is called an <b>Event Timer</b>

In [ ]:
from tkinter import *
from tkinter import ttk

def start():
    button.configure(text='Stop', command=stop)
    label['text'] = 'Working...'
    global interrupt; interrupt = False
    root.after(1, step)

def stop():
    global interrupt; interrupt = True

def step(count=0):
    progress['value'] = count
    if interrupt:
        result(None)
        return
    root.after(100)
    if count==20:
        result(42)
        return
    root.after(1, lambda: step(count+1))
    
def result(answer):
    progress['value'] = 0
    button.configure(text='Start!', command=start)
    label['text'] = 'Answer: ' + str(answer) if answer else "No Answer"

root = Tk()

frame = ttk.Frame(root); frame.grid()
button = ttk.Button(frame, text="Start!", command=start); button.grid(column=1, row=0, padx=5, pady=5)
label = ttk.Label(frame, text='No Answer'); label.grid(column=0, row=0, padx=5, pady=5)
progress = ttk.Progressbar(frame, orient='horizontal', mode="determinate", maximum=20)
progress.grid(column=0, row=1, padx=5, pady=5)

root.mainloop()

<h2>Asynchronous I/O<h2>

Timer events take care of braking upa  long-running computation, where you know that each step can be guaranteed to complete quickly so that your handler will return to the event loop.<br>
Some operations dont finish quickly such as communicating witha  databse or retrieving data from a remote web server.<br>
Most I/O calls are <i>blocking</i> what we want to do is use a <i>non-blocking</i> or <i>asynchronous I/O</i> call, that way it returns to the to the event loop immediately and the when the I/O operation completes, your program is notified and can process the result of the I/O operation.<br>
This is called a <i>event-drive I/O</i>.

asynchronous I/O is provided by the `asyncio` module.<br>
Best practice is to keep Tkinter event loop running in the main thread and spin-off your asyncio event loops in another thread.<br>
The applicaion code, running in the main thread, may need to coordinate with asyncio event loop running in the other thread. You can call a function runnin in the asyncio event loop thread(even from the Tkinter event loop in a widget callback) using the asyncio `call_soon_threadsafe` method.

<h3>Threads or Processes</h3>

main rule is that you must only make Tk calls from the thread where you loaded Tk.<br>
if you need to communicate from another thread to the thread running Tkinter, keep it as simple as possible. Use `event_generate` to post a virtual event to the Tkinter event queue, and then `bind` to that event in your code.<br><br>
`root.event_generate("<<MyOwnEvent>>")`<br><br>
If you have more than one thread in your application, make sure you're running in a threaded build. If you're unsure, check the Tcl variable `tcl_platform(threaded)`; it should be 1, not 0.

`tkinter.Tcl().eval('set tcl_platform(threaded)')`


<h1>Example of Threading in Practice</h1>

In [6]:
from tkinter import *
from tkinter import ttk
import time
from random import randint
import threading

root = Tk()

root.title("Threading example")
root.geometry("500x400")

def five_seconds():
    time.sleep(5)
    my_label['text'] = "5 seconds is Up!"

def rando():
    random_label['text'] = f"Random Number = {randint(1,100)}"

my_label = ttk.Label(root, text="Hello There!")
my_label.grid(pady=20, padx=200, sticky=(W,E))

my_button = ttk.Button(root, text='5 Seconds', 
                       command=threading.Thread(target=five_seconds).start())
my_button.grid(pady=20, padx=200, sticky=(W,E))

my_button2 = ttk.Button(root, text="Pick Random Number", command=rando)
my_button2.grid(pady=20, padx=200, sticky=(W,E))

random_label = ttk.Label(root, text='')
random_label.grid(pady=20, padx=200, sticky=(W,E))


root.mainloop()

<h2>Nested Event Processing</h2>

Within your long-running operation, you can invoke the event loop to process a bunch of events. You can do this with a single command, `update`. There's no messing around with timer evens or asynchronou I/O. Just sprinkle some `update` calls throught your operation. If you want to only keep the screen redrawing but not process other events, there's even a option for that(`update_idletasks`)<br><br>
This approach is seductively easy. And if you're lucky, it might work. At least for a little while. But sooner or later, you're going to run into serious difficulties trying to do things that way. Something won't be updating, event handlers aren't getting called that should be, events are going missing or being fired out of order, or worse. You'll turn your program's logic inside out and tear your hair out trying to make it work again.<br><br>
In practice your askign for trouble if you use `udpate` as it is not giving control to the eventloop is is essentually making a new event loop inside of the existing one and this can get messy really quick